# `CLOUD.jl` - 2D linear advection example

In [15]:
using CLOUD, OrdinaryDiffEq

## Define the physical problem

In [16]:
θ = π/4 # wave angle
a = sqrt(2) # wave speed
A = 1.0  # amplitude
L = 1.0  # domain length
k = (2*π/L, 2*π/L)  # wave number
T = 1.0/(a*max(abs(cos(θ)),abs(sin(θ)))) # end time for one period

conservation_law = LinearAdvectionEquation((a*cos(θ),a*sin(θ)))
initial_data = InitialDataSine(A,k)
exact_solution = ExactSolution(conservation_law,initial_data);

## Set discretization parameters (energy-conservative scheme)

In [17]:
M = 4
p = 4
p_map = 4
form = WeakConservationForm(mapping_form=SkewSymmetricMapping(), 
    inviscid_numerical_flux=LaxFriedrichsNumericalFlux(0.0))
strategy = ReferenceOperator()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.01*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

## Set up a `NodalTensor` scheme on quadrilaterals of degree `p` with `M` edges in each direction

In [18]:
reference_approximation = ReferenceApproximation(
    NodalTensor(p), Quad(), mapping_degree=p_map)

mesh = warp_mesh(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    ((0.0,L),(0.0,L)), (M,M)), 
    reference_approximation.reference_element, 0.2)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), strategy,
     "results/advection_2d/", overwrite=true, clear=true);

## Run the solver

In [19]:
ode_problem_1 = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy);

save_solution(ode_problem_1.u0, 0.0, results_path, 0)

CLOUD_reset_timer!()
sol = solve(ode_problem_1, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")

 ────────────────────────────────────────────────────────────────────────────────
            Thread 1                    Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            10.2s /  91.1%           10.2GiB /  97.3%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 local residual          407k    8.60s   92.4%  21.1μs   9.48GiB   95.3%  24.4KiB
   apply operators       407k    4.40s   47.2%  10.8μs   5.78GiB   58.1%  14.9KiB
     volume terms        407k    2.07s   22.3%  5.09μs   4.20GiB   42.2%  10.8KiB
     mass matrix solve   407k    427ms    4.6%  1.05μs    199MiB    2.0%     512B
     facet terms         407k    399ms    4.3%   979ns    566MiB    5.6%  1.42KiB
     mul test function   407k    159ms    1.7%   389ns    218MiB    2.1%     560B
   gather ext s

## Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [20]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data, T)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.07575412332576625
Conservation (initial/final/diff):
[-1.033172068290629e-5][-1.033172068353079e-5][-6.245004513516506e-16]
Energy (initial/final/diff):
[0.12499380403835125][0.12499380403835086][-3.885780586188048e-16]


## Write output to VTK file

In [21]:
postprocess_vtk(spatial_discretization, string(results_path,"solution.vtu"), last(sol.u));